# 07 draft strategy Notebook

This notebook implements the analysis for the 07 draft strategy stage of the Fantasy Football Analysis project.

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os

# Add the src directory to path
sys.path.append('..')

# Set display options
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

In [ ]:
# TODO: Implement 07 draft strategy analysis

In [ ]:
# Import project modules
from src.data.data_loader import load_config
from src.data.data_loader mport load_csv_data
from src.visualization.draft_strategy_vis import (
    plot_draft_strategy_heatmap,
    plot_position_value_chart,
    plot_target_player_tiers
)
from src.utils.validation import validate_analysis_output

# Set pandas display options
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

In [ ]:
# 1. Load configuration and processed data
config = load_config()

# Load key analysis results
vbd_df = load_csv_data(os.path.join(config['data_paths']['processed_data'], 'vbd_rankings.csv'))
round_stats = load_csv_data(os.path.join(config['data_paths']['processed_data'], 'round_stats.csv'))
position_round_stats = load_csv_data(os.path.join(config['data_paths']['processed_data'], 'position_round_stats.csv'))
optimal_positions = load_csv_data(os.path.join(config['data_paths']['processed_data'], 'optimal_positions_by_round.csv'))

# Load tier and archetype data
tier_data = {}
archetype_data = {}
for pos in ['QB', 'RB', 'WR', 'TE']:
    try:
        tier_data[pos] = load_csv_data(os.path.join(config['data_paths']['processed_data'], f"{pos}_tiers.csv"))
        archetype_data[pos] = load_csv_data(os.path.join(config['data_paths']['processed_data'], f"{pos}_archetypes.csv"))
    except:
        print(f"No tier or archetype data found for {pos}")

# Load efficiency data
try:
    efficiency_outliers = load_csv_data(os.path.join(config['data_paths']['processed_data'], 'efficiency_outliers.csv'))
    composite_efficiency = load_csv_data(os.path.join(config['data_paths']['processed_data'], 'composite_efficiency_scores.csv'))
except:
    print("No efficiency data found")
    efficiency_outliers = pd.DataFrame()
    composite_efficiency = pd.DataFrame()

In [ ]:
# 2. Create positional value overview
print("Positional Value Overview:")
position_value = vbd_df.groupby('FantPos').agg({
    'VBD': ['mean', 'sum', 'max', 'count'],
    'Half_PPR': ['mean', 'sum', 'max']
}).reset_index()
position_value.columns = ['_'.join(col).strip('_') if isinstance(col, tuple) else col for col in position_value.columns]
display(position_value)


In [ ]:
# 3. Identify optimal draft rounds by position
print("\nOptimal Draft Rounds by Position:")
display(optimal_positions)

# Calculate success rates by position and round
success_by_pos_round = position_round_stats.pivot_table(
    index='Draft_Round', 
    columns='Position', 
    values='Success_Rate'
).reset_index()
print("\nSuccess Rate by Position and Round (%):")
display(success_by_pos_round)

In [ ]:
# 4. Create draft strategy framework
print("\nDraft Strategy Framework:")

# Define rounds and pick ranges
rounds = range(1, 13)  # 12 team league, 12 rounds
pick_ranges = [(1, 4), (5, 8), (9, 12)]  # Early, middle, late picks

# Create strategy table
strategy_rows = []

for rnd in rounds:
    # Get optimal position for this round
    opt_pos = optimal_positions[optimal_positions['Draft_Round'] == rnd]['Optimal_Position'].values
    opt_pos = opt_pos[0] if len(opt_pos) > 0 else "Multiple"
    
    # Get success rates for this round
    round_success = position_round_stats[position_round_stats['Draft_Round'] == rnd]
    
    for pick_range in pick_ranges:
        start, end = pick_range
        overall_picks = [(rnd - 1) * 12 + p for p in range(start, end + 1)]
        
        # Snake draft adjustment for even rounds
        if rnd % 2 == 0:
            overall_picks = [(rnd - 1) * 12 + (13 - p) for p in range(start, end + 1)]
        
        # Determine recommended positions based on optimal position and success rates
        if not round_success.empty:
            # Get position with highest success rate
            best_pos = round_success.sort_values('Success_Rate', ascending=False)['Position'].values
            best_pos = best_pos[0] if len(best_pos) > 0 else opt_pos
            
            # Get alternative positions
            alt_positions = round_success.sort_values('Success_Rate', ascending=False)['Position'].values
            alt_positions = list(alt_positions[1:3]) if len(alt_positions) > 2 else list(alt_positions[1:]) if len(alt_positions) > 1 else []
            
            # Adjust recommendations based on pick range
            if start <= 4:  # Early picks
                primary_rec = best_pos
            elif start <= 8:  # Middle picks
                primary_rec = best_pos if best_pos == opt_pos else opt_pos
            else:  # Late picks
                primary_rec = opt_pos
        else:
            primary_rec = opt_pos
            alt_positions = []
        
        strategy_rows.append({
            'Round': rnd,
            'Pick_Range': f"{start}-{end}",
            'Overall_Picks': f"{min(overall_picks)}-{max(overall_picks)}",
            'Primary_Position': primary_rec,
            'Alternative_Positions': ', '.join(alt_positions) if alt_positions else "Any"
        })

strategy_df = pd.DataFrame(strategy_rows)
display(strategy_df)

In [ ]:
# 5. Identify target players by round and position
print("\nTarget Players by Position and Tier:")
target_players = {}

for pos in ['QB', 'RB', 'WR', 'TE']:
    if pos in tier_data and not tier_data[pos].empty:
        # Add round information based on ADP
        tier_df = tier_data[pos]
        if 'ADP' in tier_df.columns:
            tier_df['Draft_Round'] = np.ceil(tier_df['ADP'] / 12).astype(int)
            
            # Group target players by tier and round
            targets = []
            for tier in tier_df['Tier'].unique():
                tier_players = tier_df[tier_df['Tier'] == tier].sort_values('Half_PPR', ascending=False)
                
                for rnd in sorted(tier_players['Draft_Round'].unique()):
                    round_players = tier_players[tier_players['Draft_Round'] == rnd]
                    
                    if not round_players.empty:
                        top_players = round_players.head(3)['Player'].tolist()
                        targets.append({
                            'Position': pos,
                            'Tier': tier,
                            'Round': rnd,
                            'Target_Players': ', '.join(top_players),
                            'Avg_Points': round_players['Half_PPR'].mean()
                        })
            
            if targets:
                target_players[pos] = pd.DataFrame(targets)

# Display target players by position
for pos, df in target_players.items():
    print(f"\n{pos} Target Players by Tier and Round:")
    display(df.sort_values(['Round', 'Tier']))

In [ ]:
# 6. Identify high-value players based on efficiency metrics
if not composite_efficiency.empty:
    print("\nHigh Value Players Based on Efficiency:")
    
    high_value_players = []
    
    for pos in ['QB', 'RB', 'WR', 'TE']:
        pos_df = composite_efficiency[composite_efficiency['Position'] == pos]
        
        if not pos_df.empty and 'ADP' in pos_df.columns:
            # Calculate efficiency to ADP ratio
            pos_df['Efficiency_to_ADP'] = pos_df['Composite_Efficiency'] / pos_df['ADP']
            
            # Get top players by efficiency relative to draft position
            top_value = pos_df.sort_values('Efficiency_to_ADP', ascending=False).head(5)
            
            for _, row in top_value.iterrows():
                high_value_players.append({
                    'Position': pos,
                    'Player': row['Player'],
                    'Team': row['Team_std'],
                    'ADP': row['ADP'] if 'ADP' in row else None,
                    'Composite_Efficiency': row['Composite_Efficiency'],
                    'Half_PPR': row['Half_PPR']
                })
    
    if high_value_players:
        high_value_df = pd.DataFrame(high_value_players)
        display(high_value_df)


In [ ]:
# 7. Identify potential breakout candidates and regression candidates
if not efficiency_outliers.empty:
    # Potential breakout candidates (low volume, high efficiency)
    print("\nPotential Breakout Candidates (Low Volume, High Efficiency):")
    breakout_candidates = efficiency_outliers[
        (efficiency_outliers['Outlier_Type'] == 'High Efficiency') & 
        (efficiency_outliers['Improvement_Potential'].fillna(False))
    ]
    if not breakout_candidates.empty:
        display(breakout_candidates[['Player', 'Position', 'Team_std', 'Half_PPR', 'Efficiency_Score']])
    
    # Regression candidates (high efficiency outliers)
    print("\nRegression Candidates (Unsustainably High Efficiency):")
    regression_candidates = efficiency_outliers[
        (efficiency_outliers['Outlier_Type'] == 'High Efficiency') & 
        (efficiency_outliers['Regression_Risk'].fillna(False))
    ]
    if not regression_candidates.empty:
        display(regression_candidates[['Player', 'Position', 'Team_std', 'Half_PPR', 'Efficiency_Score']])

In [ ]:
# 8. Create draft cheat sheet
print("\nDraft Cheat Sheet Template:")

# Create a template for draft day
cheat_sheet = []

for rnd in range(1, 13):
    # Get recommendations for this round
    round_strategy = strategy_df[strategy_df['Round'] == rnd]
    
    # Get target players for this round
    round_targets = {}
    for pos, df in target_players.items():
        if not df.empty:
            pos_round_targets = df[df['Round'] == rnd]['Target_Players'].tolist()
            if pos_round_targets:
                round_targets[pos] = ', '.join(pos_round_targets)
    
    # Create entry for each pick range
    for _, strat in round_strategy.iterrows():
        cheat_entry = {
            'Round': rnd,
            'Pick_Range': strat['Pick_Range'],
            'Overall_Picks': strat['Overall_Picks'],
            'Primary_Position': strat['Primary_Position'],
            'Alternative_Positions': strat['Alternative_Positions'],
        }
        
        # Add target players
        for pos in ['QB', 'RB', 'WR', 'TE']:
            cheat_entry[f"{pos}_Targets"] = round_targets.get(pos, '')
        
        cheat_sheet.append(cheat_entry)

cheat_df = pd.DataFrame(cheat_sheet)
display(cheat_df)

In [ ]:
# 9. Create visualizations
# Draft strategy heatmap
plt.figure(figsize=(12, 8))
plot_draft_strategy_heatmap(position_round_stats)
plt.tight_layout()
plt.savefig('../outputs/figures/draft_strategy_heatmap.png')
plt.show()

# Position value chart
plt.figure(figsize=(12, 8))
plot_position_value_chart(vbd_df)
plt.tight_layout()
plt.savefig('../outputs/figures/position_value_chart.png')
plt.show()

# Target player tiers
plt.figure(figsize=(14, 10))
plot_target_player_tiers(target_players)
plt.tight_layout()
plt.savefig('../outputs/figures/target_player_tiers.png')
plt.show()

In [ ]:
# 10. Save draft strategy data
os.makedirs(os.path.join(config['data_paths']['processed_data'], 'draft_strategy'), exist_ok=True)
strategy_df.to_csv(os.path.join(config['data_paths']['processed_data'], 'draft_strategy', 'strategy_framework.csv'), index=False)
cheat_df.to_csv(os.path.join(config['data_paths']['processed_data'], 'draft_strategy', 'draft_cheat_sheet.csv'), index=False)

for pos, df in target_players.items():
    df.to_csv(os.path.join(config['data_paths']['processed_data'], 'draft_strategy', f"{pos}_targets.csv"), index=False)

print("Draft strategy framework completed!")